<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_29_Explore_GitHub_Repo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to explore a GitHub repository with Graphlit, and the large context window of Google Gemini. Here we show how to use 'augmented content' to bring in the entire code base as context for the RAG prompt.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

---

In [1]:
!pip install --upgrade graphlit-client

Clone the repo you want to explore

In [2]:
!rm -rf crewAI
!git clone https://github.com/crewAIInc/crewAI/

Cloning into 'crewAI'...
remote: Enumerating objects: 10554, done.
remote: Counting objects: 100% (4221/4221), done.
remote: Compressing objects: 100% (1062/1062), done.
remote: Total 10554 (delta 3110), reused 3846 (delta 2925), pack-reused 6333 (from 1)
Receiving objects: 100% (10554/10554), 26.17 MiB | 11.81 MiB/s, done.
Resolving deltas: 100% (6989/6989), done.


Initialize Graphlit

In [3]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize sample data

In [4]:
DATA_DIRECTORY = "crewAI"

PATH_CODE = DATA_DIRECTORY + "/src/crewai/"

Define Graphlit helper functions

In [5]:
import base64
import mimetypes
import asyncio
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_file(filename: str, progress_bar, workflow_id: Optional[str] = None):
    print(f'Ingesting content from [{filename}]')

    content_id = await ingest_file(filename, workflow_id)

    if content_id is not None:
        print(f'Ingested content [{content_id}] from [{filename}].')
    else:
        print(f'Failed to ingest content from [{filename}].')

    progress_bar.update(1)

# NOTE: for local files, load from disk and convert to Base64 data
async def ingest_file(file_path: str, workflow_id: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        file_name = os.path.basename(file_path)
        content_name, _ = os.path.splitext(file_name)

        if file_name[0] == ".":
            print(f'Skipping dot file [{file_name}]')
            return None

        mime_type = mimetypes.guess_type(file_name)[0]

        if mime_type is None:
            print(f'Failed to infer MIME type [{file_name}], treat as plain text')
            mime_type = "text/plain"

        if not os.path.exists(file_path):
            print(f'File [{file_path}] does not exist')
            return None

        with open(file_path, "rb") as file:
            file_content = file.read()

            if file_content is not None and len(file_content) > 0:
                base64_content = base64.b64encode(file_content).decode('utf-8')

                # Using synchronous mode, so the notebook waits for the content to be ingested
                response = await graphlit.client.ingest_encoded_file(name=content_name, data=base64_content, mime_type=mime_type,
                                                                    workflow=input_types.EntityReferenceInput(id=workflow_id) if workflow_id is not None else None, is_synchronous=True)

                return response.ingest_encoded_file.id if response.ingest_encoded_file is not None else None
            else:
                print(f'Ignoring empty file [{file_path}].')
                return None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_google_specification(model: enums.GoogleModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"Google [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.GOOGLE,
        google=input_types.GoogleModelPropertiesInput(
            model=model,
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.CONTENT, # NOTE: load entire code file, for any search hits
            contentLimit=100
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        ),
        # NOTE: bring in all ingested code as context for the conversation, no matter if it was relevant to the user prompt
        augmentedFilter=input_types.ContentCriteriaInput(
            types=[enums.ContentTypes.FILE],
            fileTypes=[enums.FileTypes.CODE]
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        return response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Ingest code repository

In [6]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

# List of files to process
files = [os.path.join(root, file) for root, _, files in os.walk(PATH_CODE) for file in files]

progress_bar = tqdm(total=len(files))

tasks = [process_file(filename, progress_bar) for filename in files]

await asyncio.gather(*tasks)

print('Ingested all contents.')

Deleted all contents.


  1%|          | 1/171 [00:00<00:20,  8.17it/s]

Ingesting content from [crewAI/src/crewai/crew.py]
Ingesting content from [crewAI/src/crewai/agent.py]
Ingesting content from [crewAI/src/crewai/__init__.py]
Ingesting content from [crewAI/src/crewai/task.py]
Ingesting content from [crewAI/src/crewai/process.py]
Ingesting content from [crewAI/src/crewai/llm.py]
Ingesting content from [crewAI/src/crewai/routers/__init__.py]
Ingesting content from [crewAI/src/crewai/routers/router.py]
Ingesting content from [crewAI/src/crewai/tasks/task_output.py]
Ingesting content from [crewAI/src/crewai/tasks/__init__.py]
Ingesting content from [crewAI/src/crewai/tasks/conditional_task.py]
Ingesting content from [crewAI/src/crewai/tasks/output_format.py]
Ingesting content from [crewAI/src/crewai/utilities/planning_handler.py]
Ingesting content from [crewAI/src/crewai/utilities/crew_json_encoder.py]
Ingesting content from [crewAI/src/crewai/utilities/i18n.py]
Ingesting content from [crewAI/src/crewai/utilities/config.py]
Ingesting content from [crewAI/s

 16%|█▌        | 27/171 [00:00<00:01, 93.27it/s]


Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/crews/write_linkedin_crew/config/agents.yaml]
Failed to infer MIME type [agents.yaml], treat as plain text
Ignoring empty file [crewAI/src/crewai/cli/templates/pipeline/crews/write_linkedin_crew/config/agents.yaml].
Failed to ingest content from [crewAI/src/crewai/cli/templates/pipeline/crews/write_linkedin_crew/config/agents.yaml].
Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/crews/write_x_crew/write_x_crew.py]
Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/crews/write_x_crew/config/tasks.yaml]
Failed to infer MIME type [tasks.yaml], treat as plain text
Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/crews/write_x_crew/config/agents.yaml]
Failed to infer MIME type [agents.yaml], treat as plain text
Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/tools/custom_tool.py]
Ingesting content from [crewAI/src/crewai/cli/templates/pipeline/tools/__init__.py]

...

Ingested content [bed43a75-6c08-4b3c-8abb-4b7edf0fb67c] from [crewAI/src/crewai/agents/agent_builder/base_agent_executor_mixin.py].
Ingested content [21698eae-c81c-4518-b5d1-a92409edb1db] from [crewAI/src/crewai/project/crew_base.py].
Ingested content [5de38605-a790-4bc6-8c1a-742d129c9b73] from [crewAI/src/crewai/telemetry/telemetry.py].


 99%|█████████▉| 170/171 [01:01<00:00,  1.19it/s]

Ingested content [5349a8a6-5526-4534-bc29-ecad321a065f] from [crewAI/src/crewai/agents/agent_builder/utilities/base_output_converter.py].


100%|██████████| 171/171 [01:04<00:00,  1.09s/it]

Ingested content [fcd6c327-6c81-4362-acdc-4bcd8b20e9fe] from [crewAI/src/crewai/project/utils.py].
Ingested all contents.


Execute example RAG prompt with citations

In [7]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

prompt = "Walk me through how the short-term and long-term memory works in CrewAI. Be detailed and show code samples."

model = enums.GoogleModels.GEMINI_1_5_FLASH
#model = enums.GoogleModels.GEMINI_1_5_PRO

specification_id = await create_google_specification(model)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id=specification_id)

    if conversation_id is not None:
        message = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown('### Conversation:'))
            display(Markdown(f'**User:**\n{prompt}'))
            display(Markdown(f'**Assistant:**\n{message}'))
            print()


Deleted all conversations and specifications.
Created specification [67193e2b-0bc7-4dd7-bf84-4f21c81d1744].


### Conversation:

**User:**
Walk me through how the short-term and long-term memory works in CrewAI. Be detailed and show code samples.

**Assistant:**
CrewAI utilizes both short-term memory (STM) and long-term memory (LTM) to enhance the performance of its agents and crews. STM stores recent insights and information gathered during the current execution of a crew, while LTM stores historical data and insights from previous executions. This allows agents to leverage past experiences and context to make more informed decisions.

The STM is implemented using the `ShortTermMemory` class, which uses a SQLite database to store recent insights. The `save` method of the `ShortTermMemory` class adds new insights to the database, while the `search` method retrieves relevant insights based on a given query.

Here's an example of how STM is used in the `ContextualMemory` class to build context for a task:

```python
from crewai.memory import ShortTermMemory

class ContextualMemory:
    def __init__(self, stm: ShortTermMemory, ltm: LongTermMemory, em: EntityMemory):
        self.stm = stm
        self.ltm = ltm
        self.em = em
    def build_context_for_task(self, task, context) -> str:
        """
        Automatically builds a minimal, highly relevant set of contextual information
        for a given task.
        """
        query = f"{task.description} {context}".strip()
        if query == "":
            return ""
        context = []
        context.append(self._fetch_ltm_context(task.description))
        context.append(self._fetch_stm_context(query))
        context.append(self._fetch_entity_context(query))
        return "\n".join(filter(None, context))
    def _fetch_stm_context(self, query) -> str:
        """
        Fetches recent relevant insights from STM related to the task's description and expected_output,
        formatted as bullet points.
        """
        stm_results = self.stm.search(query)
        formatted_results = "\n".join([f"- {result}" for result in stm_results])
        return f"Recent Insights:\n{formatted_results}" if stm_results else ""
```

The LTM is implemented using the `LongTermMemory` class, which also uses a SQLite database to store historical data. The `save` method of the `LongTermMemory` class adds new data to the database, while the `search` method retrieves relevant data based on a given query.

Here's an example of how LTM is used in the `ContextualMemory` class to build context for a task:

```python
from crewai.memory import LongTermMemory

class ContextualMemory:
    def __init__(self, stm: ShortTermMemory, ltm: LongTermMemory, em: EntityMemory):
        self.stm = stm
        self.ltm = ltm
        self.em = em
    def build_context_for_task(self, task, context) -> str:
        """
        Automatically builds a minimal, highly relevant set of contextual information
        for a given task.
        """
        query = f"{task.description} {context}".strip()
        if query == "":
            return ""
        context = []
        context.append(self._fetch_ltm_context(task.description))
        context.append(self._fetch_stm_context(query))
        context.append(self._fetch_entity_context(query))
        return "\n".join(filter(None, context))
    def _fetch_ltm_context(self, task) -> Optional[str]:
        """
        Fetches historical data or insights from LTM that are relevant to the task's description and expected_output,
        formatted as bullet points.
        """
        ltm_results = self.ltm.search(task, latest_n=2)
        if not ltm_results:
            return None
        formatted_results = [
            suggestion
            for result in ltm_results
            for suggestion in result["metadata"]["suggestions"]  # type: ignore # Invalid index type "str" for "str"; expected type "SupportsIndex | slice"
        ]
        formatted_results = list(dict.fromkeys(formatted_results))
        formatted_results = "\n".join([f"- {result}" for result in formatted_results])  # type: ignore # Incompatible types in assignment (expression has type "str", variable has type "list[str]")
        return f"Historical Data:\n{formatted_results}" if ltm_results else ""
```

In summary, STM and LTM work together to provide agents with a comprehensive understanding of past and present information, enabling them to make more informed decisions and improve their performance over time.